In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
attr = {'Goalkeeping': 'keepersadv',
              'Shooting': 'shooting',
              'Passing': 'passing',
              'Pass types': 'pass_types',
              'Goal and Shot Creation': 'gca',
              'Defensive Actions': 'defense',
              'Possession': 'possession',
              'Playing Time': 'playingtime',
              'Miscellaneous Stats': 'misc'
             }

In [3]:
Leagues = { 'Serie A': ['https://fbref.com/en/comps/11/2022-2023/', '/2022-2023-Serie-A-Stats'],
            'Premier League': ['https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats'],
            'La Liga': ['https://fbref.com/en/comps/12/2022-2023/','/2022-2023-La-Liga-Stats'],
            'Bundesliga': ['https://fbref.com/en/comps/20/2022-2023/','/2022-2023-Bundesliga-Stats'],
            'Ligue 1': ['https://fbref.com/en/comps/13/2022-2023/','/2022-2023-Ligue-1-Stats']}

In [70]:
# Existing data
League = {'Serie A': {'AC Milan' : 'ACM'}}

# Add Juventus to Serie A
League['Serie A']['Juventus'] = 'JUV'

print(League)


{'Serie A': {'AC Milan': 'ACM', 'Juventus': 'JUV'}}


In [11]:
#Goalkeeping Data for the leagues
for j in Leagues.keys():
    url = Leagues[j][0] + attr['Goalkeeping'] + Leagues[j][1]
    print(url)

https://fbref.com/en/comps/11/2022-2023/keepersadv/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/9/2022-2023/keepersadv/2022-2023-Premier-League-Stats
https://fbref.com/en/comps/12/2022-2023/keepersadv/2022-2023-La-Liga-Stats
https://fbref.com/en/comps/20/2022-2023/keepersadv/2022-2023-Bundesliga-Stats
https://fbref.com/en/comps/13/2022-2023/keepersadv/2022-2023-Ligue-1-Stats


In [12]:
print(response.status_code)

200


In [69]:
url = "https://fbref.com/en/comps/11/2022-2023/2022-2023-Serie-A-Stats"

response = requests.get(url)
SerieA = {}
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Select the table containing player statistics
    table = soup.select('#results2022-2023111_overall tbody')
    # Check if the table is found
    if table:
        # Convert the table to string and create a new BeautifulSoup object
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        
        # Find all table rows within the table
        clubs = table_soup.find_all('tr')
        
        # Iterate over each row and extract player information
        for row in clubs:
            # Extract player name
            Club = row.find('td', {'data-stat': 'team'}).text.strip()

            link = 'https://fbref.com/' + row.find('a')['href']

            # Print or store the extracted information
            SerieA[Club] = link
    else:
        print("Table not found on the page.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)
print(SerieA)

{'Napoli': 'https://fbref.com//en/squads/d48ad4ff/2022-2023/Napoli-Stats', 'Lazio': 'https://fbref.com//en/squads/7213da33/2022-2023/Lazio-Stats', 'Inter': 'https://fbref.com//en/squads/d609edc0/2022-2023/Internazionale-Stats', 'Milan': 'https://fbref.com//en/squads/dc56fe14/2022-2023/Milan-Stats', 'Atalanta': 'https://fbref.com//en/squads/922493f3/2022-2023/Atalanta-Stats', 'Roma': 'https://fbref.com//en/squads/cf74a709/2022-2023/Roma-Stats', 'Juventus': 'https://fbref.com//en/squads/e0652b02/2022-2023/Juventus-Stats', 'Fiorentina': 'https://fbref.com//en/squads/421387cf/2022-2023/Fiorentina-Stats', 'Bologna': 'https://fbref.com//en/squads/1d8099f8/2022-2023/Bologna-Stats', 'Torino': 'https://fbref.com//en/squads/105360fe/2022-2023/Torino-Stats', 'Monza': 'https://fbref.com//en/squads/21680aa4/2022-2023/Monza-Stats', 'Udinese': 'https://fbref.com//en/squads/04eea015/2022-2023/Udinese-Stats', 'Sassuolo': 'https://fbref.com//en/squads/e2befd26/2022-2023/Sassuolo-Stats', 'Empoli': 'https

In [45]:
# Send a GET request to the webpage
url = 'https://fbref.com/en/squads/dc56fe14/2023-2024/all_comps/Milan-Stats-All-Competitions#all_stats_defense'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Select the table containing player statistics
    table = soup.select('table#stats_standard_combined tbody')
    # Check if the table is found
    if table:
        # Convert the table to string and create a new BeautifulSoup object
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        
        # Find all table rows within the table
        player_rows = table_soup.find_all('tr')
        
        # Iterate over each row and extract player information
        for row in player_rows:
            # Extract player name
            player_name = row.find('th', {'data-stat': 'player'}).text.strip()
            
            # Extract other statistics
            nationality = row.find('td', {'data-stat': 'nationality'}).text.strip()
            position = row.find('td', {'data-stat': 'position'}).text.strip()
            age = row.find('td', {'data-stat': 'age'}).text.strip()
            games = row.find('td', {'data-stat': 'games'}).text.strip()
            goals = row.find('td', {'data-stat': 'goals'}).text.strip()
            assists = row.find('td', {'data-stat': 'assists'}).text.strip()
            
            # Print or store the extracted information
            print(f"Player: {player_name}, Nationality: {nationality}, Position: {position}, Age: {age}, Games: {games}, Goals: {goals}, Assists: {assists}")
    else:
        print("Table not found on the page.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)

Player: Tijjani Reijnders, Nationality: nl NED, Position: MF, Age: 25-198, Games: 31, Goals: 2, Assists: 2
Player: Theo Hernández, Nationality: fr FRA, Position: DF, Age: 26-129, Games: 28, Goals: 3, Assists: 6
Player: Mike Maignan, Nationality: fr FRA, Position: GK, Age: 28-224, Games: 27, Goals: 0, Assists: 1
Player: Davide Calabria, Nationality: it ITA, Position: DF, Age: 27-068, Games: 28, Goals: 0, Assists: 4
Player: Christian Pulisic, Nationality: us USA, Position: FW, Age: 25-147, Games: 29, Goals: 7, Assists: 6
Player: Rafael Leão, Nationality: pt POR, Position: FW, Age: 24-247, Games: 27, Goals: 6, Assists: 6
Player: Olivier Giroud, Nationality: fr FRA, Position: FW, Age: 37-135, Games: 27, Goals: 12, Assists: 10
Player: Fikayo Tomori, Nationality: eng ENG, Position: DF, Age: 26-055, Games: 22, Goals: 3, Assists: 0
Player: Ruben Loftus-Cheek, Nationality: eng ENG, Position: MF, Age: 28-020, Games: 24, Goals: 5, Assists: 1
Player: Yunus Musah, Nationality: us USA, Position: MF,

In [40]:
import pandas as pd

# Assuming df is your DataFrame
# For demonstration purposes, let's create a sample DataFrame
data = {'Name': ['John', 'Emma', 'Ryan'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Paris', 'London']}
df = pd.DataFrame(data)

# Export the DataFrame to a CSV file
df.to_csv('output.csv', index=False)


In [ ]:
url = "https://fbref.com/en/comps/13/2022-2023/keepersadv/2022-2023-Ligue-1-Stats#all_stats_keeper_adv"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.select('table#stats_keeper_adv tbody')
    print(table)

else:
    print("Failed to retrieve data. Status code:", response.status_code)